A link to the "baby" CORA dataset. Not used and should be the same as the included `.tsv` files from https://relational.fit.cvut.cz/dataset/CORA.

In [1]:
#!wget http://www.cs.umd.edu/~sen/lbc-proj/data/cora.tgz

The is the full CORA dataset version 1.0.

In [6]:
from pathlib import Path

!wget -N http://people.cs.umass.edu/~mccallum/data/cora-classify.tar.gz
!tar --skip-old-files -zxf cora-classify.tar.gz
CORA_PATH = Path('cora')

--2023-02-01 08:09:43--  http://people.cs.umass.edu/~mccallum/data/cora-classify.tar.gz
Resolving people.cs.umass.edu (people.cs.umass.edu)... 128.119.240.99
Connecting to people.cs.umass.edu (people.cs.umass.edu)|128.119.240.99|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://people.cs.umass.edu/~mccallum/data/cora-classify.tar.gz [following]
--2023-02-01 08:09:43--  https://people.cs.umass.edu/~mccallum/data/cora-classify.tar.gz
Connecting to people.cs.umass.edu (people.cs.umass.edu)|128.119.240.99|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘cora-classify.tar.gz’ not modified on server. Omitting download.



## Process `papers`

Find all of the reference fields.

In [35]:
import re

# Open the file
tag_set = set()

with open(filename, 'r') as file:
    for i, line in enumerate(file):
        # Use a regular expression to find all tags in the line
        tags = re.findall(r'</(.*?)>', line)
        tag_set |= set(tags)

print(tag_set)

# Make a reasonable ordering
all_tags = ['author', 'title', 'type', 'institution', 'booktitle', 'publisher', 'editor', 'address', 'journal', 'volume', 'pages', 'month', 'year', 'note']

assert set(all_tags) == set(tag_set)

{'month', 'author', 'title', 'journal', 'editor', 'type', 'pages', 'booktitle', 'note', 'address', 'volume', 'publisher', 'institution', 'year'}


Process the entries of `papers` one line at a time, ignoring duplicate entries which have the same id.

In [83]:
import pandas as pd

# Initialize an empty dataframe
df = pd.DataFrame(columns=['id', 'filename', 'reference'] + all_tags) #, index=['id'])

missing_val = ""

last_id = None
with open(CORA_PATH / PAPERS_FILE, 'r') as file:
    # Loop through each line in the file
    for i, line in enumerate(file):
        #if i >= 2:
        #    break
        
        parts = line.strip().split("\t")
        
        # Many of the entries only have 2 parts. Ignore these
        if len(parts) != 3:
            continue
        
        id = parts[0]
        # Skip repeated ids
        if id == last_id:
            continue
        last_id = id
        
        filename = parts[1]
        
        # the first group matches a citation, e.g [B & G] if present
        try:
            m = re.match(r'\[(.+)\] (.+)', parts[2])
        except IndexError as e:
            print(f"Error {e} on entry {i}. Parts {parts}")
            raise e

        if m is None:
            reference = missing_val
            tagged_list = parts[2]
            #print(f'entry: {i}, {parts[2]}')
        else:
            reference = m.group(1)
            tagged_list = m.group(2)
        
        #reference = re.search(r'[(.*?)]', parts[2])
        #reference = missing_val if reference is None else reference

        #reference = parts[2].split(" ")[0].strip("[]")
        
        try:
            #details = parts[2].split("]")[1].strip().split("<")[1:]
            details = tagged_list.split("<")[1:]
            details = [x.split(">") for x in details]
            details = {x[0]: x[1].strip() for x in details}
        except IndexError as e:
            print(f"Error {e} on entry {i}. Parts {parts}")
            raise e
        
        row = {'id': id, 'filename': filename, 'reference': reference}
        
        for i, tag in enumerate(all_tags):
            row[tag] = details.get(tag, missing_val).rstrip(".,")
        
        # author = details.get("author", "")
        # title = details.get("title", "")
        # publisher = details.get("publisher", "")
        # address = details.get("address", "")
        # year = details.get("year", "")
        # Add the values to the dataframe
        df = df.append(row, ignore_index=True)
        # {'number': number, 'url': url, 'reference': reference, 
        #                'author': author, 'title': title, 'publisher': publisher, 
        #                'address': address, 'month': month, 'year': year}, ignore_index=True)

df.set_index('id', inplace=True)
# Print the dataframe
display(df)

,filename,reference,author,title,type,institution,booktitle,publisher,editor,address,journal,volume,pages,month,year,note
id,,,,,,,,,,,,,,,,
2,http:##dimacs.rutgers.edu#techps#1994#94-07.ps,Gar,M.R. Garey & D.S. Johnson,Computers and Intractibility: A Guide to the T...,,,,Freeman,,New York,,,,,1979,
16,http:##www.cs.wisc.edu#~fischer#ftp#pub#tech-r...,DeWitt90,"D. DeWitt, P. Futtersack, D. Maier, F. Velez","""A Study of Three Alternative Workstation-Serv...",,,Proceedings of the 16th International Conferec...,,,"Brisbane, Australia",,,,August,1990,
18,ftp:##ftp.cs.purdue.edu#pub#hosking#papers#oop...,Hoski93a,"A. Hosking, J. E. B. Moss","""Object Fault Handling for Persistent Programm...",,,Proceedings of the 16th International Conferec...,,,,,,pp. 288-303,,1993,
20,http:##www.pmg.lcs.mit.edu#papers#dist-mgmt.ps.gz,Liskov93,"Liskov B., Day M., Shrira L",Distributed Object Management in Thor,,,Distributed Object Management,,In M. Tamer Ozsu and Umesh Dayal and Patrick V...,"San Mateo, California",,,,,1993,
22,http:##www.pmg.lcs.mit.edu#papers#osdi94-opplo...,Otoole94,"J. O\'Toole, L. Shrira","""Opportunistic Log: Efficient Installation Rea...",,,USENIX Symposium on Operating Systems Design a...,,,,,,pp. 39-48,November,1994,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102114,http:##wwwpub.utdallas.edu#~herve#abdi.josa.ps,29,"Valentin, D. and Abdi, H","""Can a linear autoassociator recognize faces f...",,,,,,,Journal of the Optical Society of America A 13,,717-724,,(1996),
1102216,www.cs.bilkent.edu.tr#~oulusoy#jss2.ps.Z,,"Cetintemel U., Zimmermann J., Ulusoy O., and B...",OBJECTIVE: A Benchmark for Object-Oriented Act...,Technical Report BU-CEIS-9610,"Bilkent University, Ankara, Turkey",,,,,,,,,1996,
1102254,http:##www.ri.cmu.edu#afs#cs#user#kseymore#htm...,3,S. F. Chen et al,Topic Adaptation for Language Modeling Using U...,,,in Proc. ICASSP\'98,,,,,Vol. 2,pp. 681-684,May 12-15,1998,


In [114]:
years = df['year']
print(f'There are {len(years)} entries')
W = [y for y in years if y == ""]
print(f'There are {len(W)} entries missing the publication year')
bad = [y for y in years if y != "" and not y.isdigit()]
print(f'There are {len(bad)} badly formed year entries')

#pd.Series([int(y) if isinstance(y, int) else None for y in years]).value_counts()
Y = [int(y) for y in years if y.isdigit()]
print(f'There are {len(Y)} valid entries')

assert len(years) == len(W) + len(bad) + len(Y)

print(bad)
pd.Series(Y).value_counts()
Z = [y for y in Y if y is not None and 1990 <= y and y <= 1999]
Z = [y for y in Y if y is not None]
len(Z)

There are 19396 entries
There are 2957 entries missing the publication year
There are 1527 badly formed year entries
There are 14912 valid entries
['1991)', '(1979)', '(1996)', '1990)', '(1985)', '1994b', '(1995)', '(1993)', '(1996a)', '(1994)', '(1995)', '1986)', '1993)', '(1995)', '1986)', '(1994)', '1990)', '(1994)', '1991)', '(1995)', '(1995)', '(1993)', '(1993)', '1995b', '(1995)', '(1995)', '(1996)', '(1978)', '(1992)', '(1996)', '(1995b)', '(1996)', '(1997)', '(1991)', '1992)', '1996)', '1994a', '(1993)', '(1995)', '(1994)', '(1995)', '(1993)', '(1994)', '1990)', '1991)', '(1988)', '(1987)', '(1991)', '1988)', '(1990)', '(1992)', '(1993)', '(1995)', '(1993)', '(1995)', '(1983)', '(1997)', '(1996)', '(1988)', '1993)', '1994)', '(1995a)', '1987)', '(1991)', '(1990)', '(1994)', '(1994)', '(1996)', '(1990)', '(1994)', '(1994)', '(1995)', '(1995)', '(1996)', '(1987)', '(1994)', '(1990)', '(1996)', '(1992)', '(1995)', '(1990)', '(1996)', '(1993)', '(1993)', '(1995)', '(1996)', '(1992)

14912

In [64]:
PAPERS_FILE = 'papers'

filename = CORA_PATH / PAPERS_FILE

with open(filename, 'r') as file:
    for i, line in enumerate(file):
        if i >= 2:
            break
        # process the line
        parts = line.strip().split("\t")
        print("Line number:", parts[0])
        print("URL:", parts[1])
        
        print("parts[2]:", parts[2])
        m = re.match(r'\[(\w+)\] (.+)', parts[2])
        print(m)
        print(m.group(1))
        print(m.group(2))

        m = re.match(r'[(\w+)] (.+)', parts[2])
        print(m.group(1))
        print(m.group(2))

        reference = parts[2].split(" ")[0].strip("[]")
        print(f'reference: {reference}')

        details = parts[2].split("]")[1].strip().split("<")[1:]
        details = [x.split(">") for x in details]
        details = {x[0]: x[1].strip() for x in details}
        print("Author(s):", details.get("author", ""))
        print("Title:", details.get("title", ""))
        print("Publisher:", details.get("publisher", ""))
        print("Address:", details.get("address", ""))
        print("Year:", details.get("year", ""))
        print("\n")
        
 

Line number: 2
URL: http:##dimacs.rutgers.edu#techps#1994#94-07.ps
parts[2]: [Gar] <author> M.R. Garey & D.S. Johnson, </author> <title> Computers and Intractibility: A Guide to the Theory of NP-Completeness, W.H. </title> <publisher> Freeman, </publisher> <address> New York, </address> <year> 1979. </year>
<re.Match object; span=(0, 231), match='[Gar] <author> M.R. Garey & D.S. Johnson, </autho>
Gar
<author> M.R. Garey & D.S. Johnson, </author> <title> Computers and Intractibility: A Guide to the Theory of NP-Completeness, W.H. </title> <publisher> Freeman, </publisher> <address> New York, </address> <year> 1979. </year>


AttributeError: 'NoneType' object has no attribute 'group'

In [20]:
# Open the file
with open(filename, 'r') as file:
    # Loop through each line in the file
    for i, line in enumerate(file):
        if i >= 5:
            break
        # Split the line by tab
        line_split = line.strip().split('\t')
        # Extract the tags from the line
        tags = [x.split('<')[1].split('>')[0] for x in line_split[3].split('<')[1:]]
        # Print the list of tags
        print(tags)


IndexError: list index out of range

In [10]:
tags = elements[2].split(' ')


In [11]:
tags

['[Gar]',
 '<author>',
 'M.R.',
 'Garey',
 '&',
 'D.S.',
 'Johnson,',
 '</author>',
 '<title>',
 'Computers',
 'and',
 'Intractibility:',
 'A',
 'Guide',
 'to',
 'the',
 'Theory',
 'of',
 'NP-Completeness,',
 'W.H.',
 '</title>',
 '<publisher>',
 'Freeman,',
 '</publisher>',
 '<address>',
 'New',
 'York,',
 '</address>',
 '<year>',
 '1979.',
 '</year>']

In [12]:
tags[2].strip('<author>').strip('</author>')

'M.R.'

In [13]:
tags[2]

'M.R.'